## Imports

In [6]:
# !pip install requests
import requests

#!pip install SPARQLWrapper

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

#!pip install fuzzywuzzy
import difflib
# !pip install python-Levenshtein
# !pip install thefuzz
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# from thefuzz import fuzz
# from thefuzz import process
import json
import csv
import time #

#!pip install rdflib
from rdflib import URIRef, BNode, Literal, Graph
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
from rdflib import Namespace, Literal, XSD

import pandas as pd

In [2]:
from endpoints import *
from functions import * 
from queries import * 

## Main reconciliation function

In [127]:

def reconciliation_function(dictionary, query, endpoint=None, variables_list=None, q_class=None): # ancillary function: we are already in a for loop with "name"
  all_items = dictionary.items()
  for item in all_items: 
    if item[0].endswith(" rec"): #and item[1] == []:  
      name_rec = item[0] 
      name = item[0].replace(" rec", "")
     # print(name, item[1])
      stop_words = list(filter(lambda x: 'stop' in x, item[1])) # checks if the word "stop" is in the list. returns a list of words containing "stop"
     # if item[1] == []: # we take only the ones without a manual alignment or comment to avoid mistakes/repetitions
      
      # wikidata api request
      if type(query) == dict and endpoint == wikidata_endpoint:
        if "stop wd" not in stop_words: # if there is, it means that the iteration has already been done
          final_query = query 
          final_query['search'] = name
          API_WD = "https://www.wikidata.org/w/api.php"
          r =    requests.get(API_WD, params = final_query).json() # json with all the classes to be filtered
          res = wikidata_reconciliation(r, endpoint, q_class)
          time.sleep(5)
          item[1].append("stop wd")
          if res[1] == 'class_match' or res[1] == 'no_class_given':
            if res[0] not in dictionary[name+" rec"]:
                c = see_candidate([name, "resulting q class is", res[0]])
                if c == 'y':
                  dictionary[name+" rec"].append(res[0])
       # elif if type(query) == dict and endpoint == iconclass_endpoint: # to be developed
       
      elif type(query) == str: 
        final_query = query.replace("toBeReplaced", name)
          # iconclass api request
        if query.startswith("https://iconclass"):
          print(name)
          final_q = final_query.replace(" ", "%20")
          if "stop ic" not in stop_words:
          #print(query)
            candidate_list = []
            response = requests.get(final_q)
            final = response.json()
            for el in final["result"]: 
              candidate_list.append(el)
            dictionary[name+" rec"].append("stop ic") # in this way, we search the term only once
            print(candidate_list)
            for code in candidate_list: 
              # querying directly the code gives the response with keywords: 
              query_code= "https://iconclass.org/"+code+".json"  
              response_code = requests.get(query_code)
              final_code = response_code.json()
              if "txt" in final_code: 
                if "en" in final_code["txt"]: 
                  desc = final_code["txt"]["en"]
                  if "kw" in final_code:
                      if "en" in final_code["kw"]:
                          to_be_checked = final_code["kw"]["en"]
                          for candidate in to_be_checked: 
                            if fuzz_ratio_mean(name, candidate) >= 85: 
                              c = see_candidate([name, str(to_be_checked), code, desc])
                              if c == 'y' and "https://iconclass.org/"+code not in dictionary[name+" rec"]: 
                                dictionary[name+" rec"].append("https://iconclass.org/"+code)
        else:
          res = sparql_query_setting(final_query, endpoint)

          for result in res["results"]["bindings"]:
            if "@en" in str(result[variables_list[0]]["value"]): # case found in getty vocabs
              term = str(result[variables_list[0]]["value"]).replace("@en", "")
            else: 
              term = str(result[variables_list[0]]["value"])
            if fuzz_ratio_mean(name, term) >= 95: 
              print(name, term)
              if result[variables_list[1]]["value"] not in dictionary[name+" rec"]: 
                dictionary[name+" rec"] = [result[variables_list[1]]["value"]]


    # result["Descr"]["value"], result["ScopeNote"]["value"], result["Type"]["value"]

       # print(result["Subject"]["value"], result["Term"]["value"], result["Parents"]["value"])


In [8]:
already_done_ic = []

## Personifications

In [54]:
# open dict
personificationDict = open_json("base-dictionaries/personificationDict.json")
personificationDict

{'Aion/Phanes as personification of eternal time': 'https://w3id.org/icon/data/aion/phanes-as-personification-of-eternal-time',
 'Aion/Phanes as personification of eternal time rec': [],
 'Amor': 'https://w3id.org/icon/data/amor',
 'Amor rec': [],
 'Amor Carnalis': 'https://w3id.org/icon/data/amor-carnalis',
 'Amor Carnalis rec': [],
 'Amor Divino': 'https://w3id.org/icon/data/amor-divino',
 'Amor Divino rec': [],
 'Ardor': 'https://w3id.org/icon/data/ardor',
 'Ardor rec': [],
 'Atlas': 'https://w3id.org/icon/data/atlas',
 'Atlas rec': [],
 'Aurora': 'https://w3id.org/icon/data/aurora',
 'Aurora rec': [],
 'Avarice': 'https://w3id.org/icon/data/avarice',
 'Avarice rec': [],
 'Beauty': 'https://w3id.org/icon/data/beauty',
 'Beauty rec': [],
 'Blind Death': 'https://w3id.org/icon/data/blind-death',
 'Blind Death rec': [],
 'Blind Night': 'https://w3id.org/icon/data/blind-night',
 'Blind Night rec': [],
 'Blind Synagogue': 'https://w3id.org/icon/data/blind-synagogue',
 'Blind Synagogue re

In [56]:
d = {"Heaven" : "icon/heaven", "Heaven rec": [], "Amor" : "icon/amor", "Amor rec" : [], "Amor Carnalis" : "icon/amor-carnalis", "Amor Divino rec" : []}
iconclass_query = "https://iconclass.org/api/search?q=toBeReplaced&lang=en&size=999&page=1&sort=rank&keys=0"

In [125]:
def search_iconclass_keyword(d, keyword):
    "makes a search on Iconclass, retrieves related Iconclass codes and then checks if the given keyword is among the Iconclass term ones. You can select the desired terms, that will be added to the reconciled terms of the initial dictionary"
    for key in d:
        if key.endswith(" rec"): 
            if d[key] == []: # we check if there is not a reconciliation already
                name_rec = key
                name = key.replace(" rec", "")
                name_q = name.replace(" ", "%20")
                print(name)
                my_query = "https://iconclass.org/api/search?q=toBeReplaced%20keyword&lang=en&size=999&page=1&sort=rank&keys=0"
                final_query = my_query.replace("toBeReplaced", name_q).replace("keyword", keyword)
                  # iconclass api request
                if name not in already_done_ic:
                    candidate_list = []
                    response = requests.get(final_query)
                    final = response.json()
                    for el in final["result"]: 
                        candidate_list.append(el)
                    already_done_ic.append(name)
                    #dictionary[name+" rec"].append("stop ic") # in this way, we search the term only once
                    #print(candidate_list)
                    for code in candidate_list: 
                      # querying directly the code gives the response with keywords: 
                        query_code= "https://iconclass.org/"+code+".json"  
                        response_code = requests.get(query_code)
                        final_code = response_code.json()
                        if "txt" in final_code: 
                            if "en" in final_code["txt"]: 
                                desc = final_code["txt"]["en"]
                                if "kw" in final_code:
                                    if "en" in final_code["kw"]:
                                        to_be_checked = final_code["kw"]["en"]
                                        if len(to_be_checked) >1 and keyword in to_be_checked:
                                            c = see_candidate([name, str(to_be_checked), code, desc])
                                            if c == 'y' and "https://iconclass.org/"+code not in d[name_rec]:
                                                d[name_rec].append("https://iconclass.org/"+code)
    return d


In [60]:
test = search_iconclass_keyword(d, "personification")

Amor
Amor Divino


In [ ]:
personificationDictRec = search_iconclass_keyword(personificationDict, "personification")

In [69]:
# store results
save = store_dict_in_csv("dictionaries/personificationDict.json", ["Personification", "Link"], personificationDictRec)

## Allegories

In [71]:
allegoryDict = open_json("base-dictionaries/allegoryDict.json")

In [ ]:
allegoryDictRec = search_iconclass_keyword(allegoryDict, "allegory")

## Poses and gestures
as the code https://iconclass.org/31A23 is "postures of the human figure", we filter results having the same beginning of the code

In [112]:
qualityDict = open_json("base-dictionaries/qualityDict.json")
# filter quality dict to have only the poses 
poseDict = {}
for key in qualityDict: 
    if "pose" in key: 
        poseDict[key] = qualityDict[key]

In [118]:
already_done_ic = []
def see_candidate(candidate_list): 
    "takes as input a list with the description of the candidate and allows a choice between yes and no" 
    confirm = input('Do you like this value?'+ str(candidate_list) + '(y/n):')
    return confirm

In [ ]:
poseDictRec = iconclass_spec_class(poseDict, "31A23", "posture")

In [46]:
# store results
save = store_dict_in_csv("dictionaries/poseDict.csv", ["Pose", "Link"], poseDictRec)

## Colors

In [114]:
# extrapolate colors from quality dict
colorDict = {}
for key in qualityDict: 
    if "color" in key: 
        colorDict[key] = qualityDict[key]

In [119]:
## colors 
colorDictRec = iconclass_spec_class(colorDict, "22C4")

black color
https://iconclass.org/api/search?q=black&lang=en&size=999&page=1&sort=rank&keys=0
['22C4(BLACK)']
Do you like this value?['black color', '22C4(BLACK)', 'colours, pigments, and paints: black'](y/n):y
blond color
https://iconclass.org/api/search?q=blond&lang=en&size=999&page=1&sort=rank&keys=0
[]
blue color
https://iconclass.org/api/search?q=blue&lang=en&size=999&page=1&sort=rank&keys=0
['22C4(BLUE)']
Do you like this value?['blue color', '22C4(BLUE)', 'colours, pigments, and paints: blue'](y/n):y
color black
https://iconclass.org/api/search?q=color%20black&lang=en&size=999&page=1&sort=rank&keys=0
[]
color green
https://iconclass.org/api/search?q=color%20green&lang=en&size=999&page=1&sort=rank&keys=0
[]
glowing colors
https://iconclass.org/api/search?q=glowings&lang=en&size=999&page=1&sort=rank&keys=0
[]
golden color
https://iconclass.org/api/search?q=golden&lang=en&size=999&page=1&sort=rank&keys=0
[]
rainbow color
https://iconclass.org/api/search?q=rainbow&lang=en&size=999&p

In [120]:
# store results
save = store_dict_in_csv("dictionaries/colorDict.csv", ["Color", "Link"], colorDictRec)

## Actions
subclass of 33


In [50]:
actionDict = open_json("base-dictionaries/actionDict.json")

In [51]:
actionDictRec = iconclass_spec_class(actionDict, "33", "relationship")

act of blessing
https://iconclass.org/api/search?q=act%20of%20blessing&lang=en&size=999&page=1&sort=rank&keys=0
[]
act of killing
https://iconclass.org/api/search?q=act%20of%20killing&lang=en&size=999&page=1&sort=rank&keys=0
[]
act of throwing
https://iconclass.org/api/search?q=act%20of%20throwing&lang=en&size=999&page=1&sort=rank&keys=0
[]
act of weighing
https://iconclass.org/api/search?q=act%20of%20weighing&lang=en&size=999&page=1&sort=rank&keys=0
[]
adoring
https://iconclass.org/api/search?q=adoring&lang=en&size=999&page=1&sort=rank&keys=0
[]
aguing
https://iconclass.org/api/search?q=aguing&lang=en&size=999&page=1&sort=rank&keys=0
[]
animating
https://iconclass.org/api/search?q=animating&lang=en&size=999&page=1&sort=rank&keys=0
[]
appearing
https://iconclass.org/api/search?q=appearing&lang=en&size=999&page=1&sort=rank&keys=0
[]
appearing in the sky
https://iconclass.org/api/search?q=appearing%20in%20the%20sky&lang=en&size=999&page=1&sort=rank&keys=0
[]
arranging her hair
https://ic

[]
fighting
https://iconclass.org/api/search?q=fighting&lang=en&size=999&page=1&sort=rank&keys=0
['33B31', '33B3', '33B32', '33B33', '33CC3211', '33C3211']
Do you like this value?['fighting', '33B31', 'fist-fight'](y/n):n
Do you like this value?['fighting', '33B3', 'fighting'](y/n):y
Do you like this value?['fighting', '33B32', 'kicking'](y/n):n
Do you like this value?['fighting', '33B33', 'slap in the face'](y/n):n
Do you like this value?['fighting', '33CC3211', "rivalry of women for the love of a man; fight for the hose; 'andouille' - CC - pursuit of a man"](y/n):n
Do you like this value?['fighting', '33C3211', "rivalry of women for the love of a man; fight for the hose; 'andouille'"](y/n):n
fighting with clubs
https://iconclass.org/api/search?q=fighting%20with%20clubs&lang=en&size=999&page=1&sort=rank&keys=0
[]
fighting with teeth
https://iconclass.org/api/search?q=fighting%20with%20teeth&lang=en&size=999&page=1&sort=rank&keys=0
[]
fighting with the force of muscular arms
https://ic

[]
laying
https://iconclass.org/api/search?q=laying&lang=en&size=999&page=1&sort=rank&keys=0
['33A15']
Do you like this value?['laying', '33A15', "laying the hand on another person's shoulder"](y/n):n
leading by hand
https://iconclass.org/api/search?q=leading%20by%20hand&lang=en&size=999&page=1&sort=rank&keys=0
[]
leaning on her shoulder
https://iconclass.org/api/search?q=leaning%20on%20her%20shoulder&lang=en&size=999&page=1&sort=rank&keys=0
[]
leaning upon
https://iconclass.org/api/search?q=leaning%20upon&lang=en&size=999&page=1&sort=rank&keys=0
[]
lending both hands to a woman
https://iconclass.org/api/search?q=lending%20both%20hands%20to%20a%20woman&lang=en&size=999&page=1&sort=rank&keys=0
[]
lending one hand to a man
https://iconclass.org/api/search?q=lending%20one%20hand%20to%20a%20man&lang=en&size=999&page=1&sort=rank&keys=0
[]
licking
https://iconclass.org/api/search?q=licking&lang=en&size=999&page=1&sort=rank&keys=0
[]
lifting
https://iconclass.org/api/search?q=lifting&lang=en&

Do you like this value?['loving', '33C2291', 'chamber-pot is emptied from above ~ chasing away (unwanted) lover'](y/n):n
Do you like this value?['loving', '33C3221', 'shaking lovers out of a tree, by throwing things'](y/n):n
making a terrific noise
https://iconclass.org/api/search?q=making%20a%20terrific%20noise&lang=en&size=999&page=1&sort=rank&keys=0
[]
making fun of
https://iconclass.org/api/search?q=making%20fun%20of&lang=en&size=999&page=1&sort=rank&keys=0
[]
mating
https://iconclass.org/api/search?q=mating&lang=en&size=999&page=1&sort=rank&keys=0
[]
mirroring herself
https://iconclass.org/api/search?q=mirroring%20herself&lang=en&size=999&page=1&sort=rank&keys=0
[]
mourning
https://iconclass.org/api/search?q=mourning&lang=en&size=999&page=1&sort=rank&keys=0
[]
not burning
https://iconclass.org/api/search?q=not%20burning&lang=en&size=999&page=1&sort=rank&keys=0
[]
nursing
https://iconclass.org/api/search?q=nursing&lang=en&size=999&page=1&sort=rank&keys=0
[]
offering
https://iconcla

[]
stealing
https://iconclass.org/api/search?q=stealing&lang=en&size=999&page=1&sort=rank&keys=0
[]
stepping on
https://iconclass.org/api/search?q=stepping%20on&lang=en&size=999&page=1&sort=rank&keys=0
[]
stirring water
https://iconclass.org/api/search?q=stirring%20water&lang=en&size=999&page=1&sort=rank&keys=0
[]
stopping
https://iconclass.org/api/search?q=stopping&lang=en&size=999&page=1&sort=rank&keys=0
[]
straying
https://iconclass.org/api/search?q=straying&lang=en&size=999&page=1&sort=rank&keys=0
[]
stretching out the left hand
https://iconclass.org/api/search?q=stretching%20out%20the%20left%20hand&lang=en&size=999&page=1&sort=rank&keys=0
[]
support oneself with
https://iconclass.org/api/search?q=support%20oneself%20with&lang=en&size=999&page=1&sort=rank&keys=0
[]
supporting
https://iconclass.org/api/search?q=supporting&lang=en&size=999&page=1&sort=rank&keys=0
[]
taking
https://iconclass.org/api/search?q=taking&lang=en&size=999&page=1&sort=rank&keys=0
['33A2', '33A21', '33C225']
D

In [52]:
# store results
save = store_dict_in_csv("dictionaries/actionDict.csv", ["Action", "Link"], actionDictRec)

## Emotions
part of 56

In [87]:
expressionalDict = open_json("base-dictionaries/expressionalQualityDict.json")

In [90]:
expressionalDictRec = iconclass_spec_class(expressionalDict, "56", "emotion")

agitated
https://iconclass.org/api/search?q=agitated&lang=en&size=999&page=1&sort=rank&keys=0
[]
agitated movement
https://iconclass.org/api/search?q=agitated%20movement&lang=en&size=999&page=1&sort=rank&keys=0
[]
amusement
https://iconclass.org/api/search?q=amusement&lang=en&size=999&page=1&sort=rank&keys=0
['56B4', '56B42', '56B41', '56BB41']
Do you like this value?['amusement', '56B4', 'Amusement'](y/n):y
Do you like this value?['amusement', '56B42', 'Coarseness, Rusticity'](y/n):n
Do you like this value?['amusement', '56B41', 'Fastidiousness, Epicurism'](y/n):n
Do you like this value?['amusement', '56BB41', 'Satiety'](y/n):n
apathetic pose of rest
https://iconclass.org/api/search?q=apathetic%20%20of%20rest&lang=en&size=999&page=1&sort=rank&keys=0
[]
backward gaze
https://iconclass.org/api/search?q=backward%20gaze&lang=en&size=999&page=1&sort=rank&keys=0
[]
beauty
https://iconclass.org/api/search?q=beauty&lang=en&size=999&page=1&sort=rank&keys=0
['56C1', '56CC1']
Do you like this va

[]
magnanimity
https://iconclass.org/api/search?q=magnanimity&lang=en&size=999&page=1&sort=rank&keys=0
[]
melancholy
https://iconclass.org/api/search?q=melancholy&lang=en&size=999&page=1&sort=rank&keys=0
[]
pain
https://iconclass.org/api/search?q=pain&lang=en&size=999&page=1&sort=rank&keys=0
['56BB1', '56BB19', '56BB12', '56BB11', '56BB13', '56BB121', '56BB131', '56F2522']
Do you like this value?['pain', '56BB1', "Pain, Sorrow, Sadness; 'Dolore', 'Dolore di Zeusi' (Ripa)"](y/n):n
Do you like this value?['pain', '56BB19', 'Consolation'](y/n):n
Do you like this value?['pain', '56BB12', "Torment; 'Tribulatione' (Ripa)"](y/n):n
Do you like this value?['pain', '56BB11', "Bitterness; 'Amaritudine' (Ripa)"](y/n):n
Do you like this value?['pain', '56BB13', "Heartache, Heart-break; 'Affanno', 'Cordoglio' (Ripa)"](y/n):n
Do you like this value?['pain', '56BB121', "Lamentation; 'Querela a Dio' (Ripa)"](y/n):n
Do you like this value?['pain', '56BB131', "Sighs; 'Sospiri' (Ripa)"](y/n):n
Do you like

## Natural elements
class 2 or 3

In [11]:
naturalDict = open_json("base-dictionaries/naturalElementDict.json")

In [ ]:
# open from a csv file the terms already done, to continue from where we stopped
done = pd.read_csv("natElDone.csv")
 
# converting column data to list
already_done_ic = done['NaturalElement'].tolist()

In [ ]:
# add objects regarding human beings
naturalDictRec = iconclass_spec_class(naturalDict, "3", "human")

In [44]:
already_done_ic = []

In [ ]:
# add objects of natural world
naturalDictRec2 = iconclass_spec_class(naturalDictRec, "2")

In [43]:
# store results
save = store_dict_in_csv("dictionaries/NaturalElementDict3.csv", ["NaturalElement", "Link"], naturalDictRec2)

In [ ]:
# now we look for objects, culture, society
naturalDictRec3 = iconclass_spec_class(naturalDictRec2, "4")

In [111]:
save = store_dict_in_csv("dictionaries/NaturalElementDict4.csv", ["NaturalElement", "Link"], naturalDictRec3)

## Named objects and Events


In [74]:
objDict = open_json("base-dictionaries/namedObjectDict.json")
eventDict = open_json("base-dictionaries/eventDict.json")

In [82]:
objDict3 = rumor_removal(objDict2)

In [ ]:
objDictRec = reconciliation_function(objDict2, iconclass_query)

In [84]:
# store results
save = store_dict_in_csv("dictionaries/namedObjectDict.csv", ["NamedObject", "Link"], objDict3)

In [ ]:
# events
eventDictRec = reconciliation_function(eventDict, iconclass_query)

## Symbols
keyword: symbol

In [121]:
symbDict = open_json("base-dictionaries/symbolDict.json")

In [123]:
symbolDict = {}
for key in symbDict: 
    new_key = key.replace("-", " ")
    symbolDict[new_key] = symbDict[key]

In [126]:
symbolDictRec = search_iconclass_keyword(symbolDict, "symbol")

a pair of masks, one of a young woman, the other of a old malevolent man worldliness, insincerity, falsehood
Angel smiling while carrying the coffin of the Pope angel glad that the soul of the Pope had beeneived among the blessed spirits
Angel wepting while carrying the coffin of the Pope angel grieved that the world had been deprived of such a man
Ape everything subhuman in man (lust, greed, gluttony, shamelessness)
Ape lower soul
Apollo Idolatry
archers without arrows tools of a force beyond their will and consciousness
Bacchus Folly
bat melancholy
beams of light   God's word
bear anger
Blind Fortune blindfolding Cupid and putting him on a sphere affection changes with luck
blowing in a cone shell generating panic terror
blowing in a tube fertilising
blue hue of heaven
book attribute of wisdom
bowl on a book wisdom
branches of a tree antidotes against sexual passion (Marriage, Prayer, Business, Abstinence)
Bull Christ
bundle of sticks unity
Cameo with a pair of winged horses and a ch

## Characters

In [ ]:
characterDict = open_json("characterDict.json")

In [ ]:
iconclass_query = "https://iconclass.org/api/search?q=toBeReplaced&lang=en&size=999&page=1&sort=rank&keys=0"
character_iconclass = reconciliation_function(characterDict, iconclass_query)

In [ ]:
characterDict2 = rumor_removal(characterDict)

In [ ]:
storage = store_in_json("character_Iconclass_aligned.json", characterDict4)

In [19]:
city_query = """SELECT ?qterm WHERE{?qterm wdt:P1667 <toBeReplaced>.}""" # verify if the resulting wd term is a link or is with wd prefix
viaf_query = """SELECT ?term WHERE{<toBeReplaced> wdt:P214 ?term.}""" 
variables = ["qterm"]

def search_wd_class(dictionary, query, variable_list, endpoint): 
  all_items = dictionary.items() # ordered dict of tuples
  for item in all_items: # tuple with key-value pairs
    if type(item[1]) == list: 
      for el in item[1]: # item[1]
        if el.startswith("http://vocab.getty.edu/tgn/"): # in this way, we also filter automatically also the " rec" keys
          cleaned_el = el.replace("http://vocab.getty.edu/tgn/", "")
          print(el, cleaned_el)
          final_query = query.replace("toBeReplaced", cleaned_el)
          final2_query = final_query.replace("<", "\"").replace(">", "\"")
          print(final2_query)
          res = sparql_query_setting(final2_query, endpoint)
          for result in res["results"]["bindings"]:
            print(result)
            for var in variable_list: 
              if var in result:
                  aligned = result[var]["value"]
                  print(aligned)
                  if aligned not in dictionary[item[0]]: 
                    dictionary[item[0]].append(aligned)
  return(dictionary)



http://vocab.getty.edu/tgn/7010588 7010588
SELECT ?qterm WHERE{?qterm wdt:P1667 "7010588".}
http://vocab.getty.edu/tgn/7006952 7006952
SELECT ?qterm WHERE{?qterm wdt:P1667 "7006952".}
{'qterm': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q727'}}
http://www.wikidata.org/entity/Q727
http://vocab.getty.edu/tgn/7006676 7006676
SELECT ?qterm WHERE{?qterm wdt:P1667 "7006676".}
{'qterm': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q116404'}}
http://www.wikidata.org/entity/Q116404
http://vocab.getty.edu/tgn/7007856 7007856
SELECT ?qterm WHERE{?qterm wdt:P1667 "7007856".}
{'qterm': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q12892'}}
http://www.wikidata.org/entity/Q12892
http://vocab.getty.edu/tgn/7006072 7006072
SELECT ?qterm WHERE{?qterm wdt:P1667 "7006072".}
http://vocab.getty.edu/tgn/7008775 7008775
SELECT ?qterm WHERE{?qterm wdt:P1667 "7008775".}
http://vocab.getty.edu/tgn/7005119 7005119
SELECT ?qterm WHERE{?qterm wdt:P1667 "7005119".}
http://vocab.

{'qterm': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q546'}}
http://www.wikidata.org/entity/Q546
http://vocab.getty.edu/tgn/7005688 7005688
SELECT ?qterm WHERE{?qterm wdt:P1667 "7005688".}
{'qterm': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q495'}}
http://www.wikidata.org/entity/Q495
http://vocab.getty.edu/tgn/7018159 7018159
SELECT ?qterm WHERE{?qterm wdt:P1667 "7018159".}
{'qterm': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q641'}}
http://www.wikidata.org/entity/Q641
http://vocab.getty.edu/tgn/7003204 7003204
SELECT ?qterm WHERE{?qterm wdt:P1667 "7003204".}
{'qterm': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6537'}}
http://www.wikidata.org/entity/Q6537
http://vocab.getty.edu/tgn/7003321 7003321
SELECT ?qterm WHERE{?qterm wdt:P1667 "7003321".}
{'qterm': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1741'}}
http://www.wikidata.org/entity/Q1741
http://vocab.getty.edu/tgn/7004335 7004335
SELECT ?qterm WHERE{?qterm wdt

In [ ]:
ulan_query = """SELECT ?term WHERE{<toBeReplaced> wdt:P245 ?term.}""" # verify if the resulting wd term is a link or is with wd prefix
viaf_query = """SELECT ?term WHERE{<toBeReplaced> wdt:P214 ?term.}""" 
variables = ["term"]

def alignments_through_wd(dictionary, query, variable_list, endpoint, url_string): 
  all_items = dictionary.items() # ordered dict of tuples
  for item in all_items: # tuple with key-value pairs
    if type(item[1]) == list: 
      for el in item[1]: # item[1]
        if el.startswith("http://www.wikidata.org/entity/"): # in this way, we also filter automatically also the " rec" keys
          print(el, item[0])
          cleaned_el = el.replace("\"","")
          final_query = query.replace("toBeReplaced", cleaned_el)
          res = sparql_query_setting(final_query, endpoint)
          for result in res["results"]["bindings"]:
            for var in variable_list: 
              aligned = url_string+result[var]["value"]
              print(aligned)
              if aligned not in dictionary[item[0]]: 
                dictionary[item[0]].append(aligned)
  return(dictionary)

        # idea: if the wd term was reconciled, than we take the ulan or viaf alignment
#ulan_aligned = alignments_through_wd(peopleDict, ulan_query, variables, wikidata_endpoint, "http://vocab.getty.edu/ulan/")

In [ ]:
# stop words and others removal. After this passage: ready to reconcile

def rumor_removal(dictionary):
  for item in dictionary.items(): 
    if type(item[1]) == list: 
      for element in item[1]: 
        if not element.startswith("http"): 
          item[1].remove(element)
  return(dictionary)

# peopleDictFinal = rumor_removal(peopleDict4)

## Characters reconciliation with Wikidata, VIAF, Getty IA, Iconclass

In [ ]:
#characterDict5 = rumor_removal(characterDict4)
print(characterDict5)
characterDict6 = rumor_removal(characterDict5)
storage = store_in_json("character_wd_first_alignment.json", characterDict6)

{'Adam': 'http://icondataset.org/adam', 'Adam rec': ['https://iconclass.org/11I62(ADAM)', 'http://www.wikidata.org/entity/Q69488', 'stop wd'], 'Aeneas': 'http://icondataset.org/aeneas', 'Aeneas rec': ['https://iconclass.org/96C(AENEAS)', 'http://www.wikidata.org/entity/Q82732', 'stop wd'], 'Aeolus': 'http://icondataset.org/aeolus', 'Aeolus rec': ['http://www.wikidata.org/entity/Q172549', 'stop wd'], 'Aglauros': 'http://icondataset.org/aglauros', 'Aglauros rec': ['http://www.wikidata.org/entity/Q368994', 'stop wd'], 'Aion/Phanes': 'http://icondataset.org/aion/phanes', 'Aion/Phanes rec': ['http://www.iconclass.org/rkd/91A14%28AION%29/', 'stop wd'], 'Anteros': 'http://icondataset.org/anteros', 'Anteros rec': ['http://www.iconclass.org/rkd/92D1911/', 'http://www.wikidata.org/entity/Q572133', 'stop wd'], 'Antiphas, son of Laoocoon': 'http://icondataset.org/antiphas-son-of-laoocoon', 'Antiphas, son of Laoocoon rec': ['stop wd'], 'Antoninus Pius': 'http://icondataset.org/antoninus-pius', 'Ant

In [ ]:
wd_character_query = { # nb:applying only to wikidata. Every API has its own parameters. 
        "action": "wbsearchentities",
        "format": "json",
        "language": "en",
        "search": "toBeReplaced"
    }


character = reconciliation_function(characterDict5, wd_character_query, wikidata_endpoint)

Apollo resulting q class is http://www.wikidata.org/entity/Q37340
Apostles resulting q class is http://www.wikidata.org/entity/Q4155679
Archangel Gabriel resulting q class is http://www.wikidata.org/entity/Q81989
Argus resulting q class is http://www.wikidata.org/entity/Q189821
Ariadne resulting q class is http://www.wikidata.org/entity/Q184874
Athene resulting q class is http://www.wikidata.org/entity/Q37122
bacchant resulting q class is http://www.wikidata.org/entity/Q2878203
Bacchus resulting q class is http://www.wikidata.org/entity/Q645312
Cacus resulting q class is http://www.wikidata.org/entity/Q754686
Cerberus resulting q class is http://www.wikidata.org/entity/Q83496
Christ resulting q class is http://www.wikidata.org/entity/Q302
Christ as a child resulting q class is http://www.wikidata.org/entity/Q104586046
crocodile resulting q class is http://www.wikidata.org/entity/Q2535664
Cybele resulting q class is http://www.wikidata.org/entity/Q188236
David resulting q class is http:

In [64]:
characterDict = open_json("character_wd_first_alignment.json")

In [77]:

# Second reconciliation: if we have the WD term, we align also ULAN and VIAF

ulan_query = """SELECT ?term WHERE{<toBeReplaced> wdt:P245 ?term.}""" # verify if the resulting wd term is a link or is with wd prefix
viaf_query = """SELECT ?term WHERE{<toBeReplaced> wdt:P214 ?term.}""" 
variables = ["term"]


        # idea: if the wd term was reconciled, than we take the ulan or viaf alignment
#ulan_aligned = alignments_through_wd(peopleDict, ulan_query, variables, wikidata_endpoint, "http://vocab.getty.edu/ulan/")

ulan_aligned = alignments_through_wd(characterDict4, ulan_query, variables, wikidata_endpoint, "http://vocab.getty.edu/ulan/")
viaf_aligned = alignments_through_wd(characterDict4, viaf_query, variables, wikidata_endpoint, "http://viaf.org/viaf/")

print(viaf_aligned)

http://www.wikidata.org/entity/Q69488 Adam rec
http://www.wikidata.org/entity/Q82732 Aeneas rec
http://www.wikidata.org/entity/Q172549 Aeolus rec
http://www.wikidata.org/entity/Q368994 Aglauros rec
http://www.wikidata.org/entity/Q572133 Anteros rec
http://www.wikidata.org/entity/Q1429 Antoninus Pius rec
http://vocab.getty.edu/ulan/500115700
http://www.wikidata.org/entity/Q37340 Apollo rec
http://www.wikidata.org/entity/Q4155679 Apostles rec
http://www.wikidata.org/entity/Q81989 Archangel Gabriel rec
http://www.wikidata.org/entity/Q189821 Argus rec
http://www.wikidata.org/entity/Q184874 Ariadne rec
http://www.wikidata.org/entity/Q37122 Athene rec
http://www.wikidata.org/entity/Q2878203 bacchant rec
http://www.wikidata.org/entity/Q645312 Bacchus rec
http://www.wikidata.org/entity/Q754686 Cacus rec
http://www.wikidata.org/entity/Q83496 Cerberus rec
http://www.wikidata.org/entity/Q302 Christ rec
http://www.wikidata.org/entity/Q2535664 crocodile rec
http://www.wikidata.org/entity/Q188236 Cy

http://viaf.org/viaf/105157340587209920668
http://viaf.org/viaf/40151776766418012003
http://viaf.org/viaf/77146513237432210217
http://www.wikidata.org/entity/Q34201 Jupiter rec
http://viaf.org/viaf/3266770
http://www.wikidata.org/entity/Q63851811 Jupiter as an eagle rec
http://www.wikidata.org/entity/Q177854 Lorenzo de' Medici rec
http://viaf.org/viaf/54169908
http://www.wikidata.org/entity/Q345 Mary rec
http://viaf.org/viaf/207159474183027661574
http://viaf.org/viaf/86913753
http://www.wikidata.org/entity/Q126916 Minerva rec
http://viaf.org/viaf/13107718
http://www.wikidata.org/entity/Q9077 Moses rec
http://viaf.org/viaf/805492
http://www.wikidata.org/entity/Q66016 Muses rec
http://viaf.org/viaf/115156010065049582325
http://viaf.org/viaf/5727734
http://www.wikidata.org/entity/Q6497135 Mithras rec
http://viaf.org/viaf/72187436
http://www.wikidata.org/entity/Q332 Neptune rec
http://viaf.org/viaf/237238941
http://www.wikidata.org/entity/Q81422 Noah rec
http://viaf.org/viaf/11187922
http:

In [80]:
save = store_in_json("Final/characterAlignedFinal.json", characterDict4)

store in csv: name, string that contains all the reconciled values separated by " @ "
use it to save as csv and filter the reconciled terms directly in the spreadsheet. In this way, the reconciliation is always up to date. 